In [12]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd
import os
from datetime import datetime

# 6 mm Aerogel Demo 
Environmental Condition:
- Outdoor
- Wind speed: 2 m/s
- Solar flux: 1 kW/m^2^ 
- Ambient temperature: 25°C
- Stage number: 6

In [13]:
# --- Input Parameters ---
params = {
    "q_sun": 1000,           # Incident solar flux (W/m^2), adjusted for aerogel's transmittance and absorber's reflection  
    "ref_abs": 0.9645,       # Solar absorber's reflectance (3.55%)   
    "Tr": 0.9463,             # 6 mm aerogel solar weighted transmittance (94.63%)
    "condition": 'indoor',  
    "T_amb_c": 25,           # Ambient air temperature (Celsius)
    "T_b_c": 30,             # Back wall temperature (Celsius), set to be 5C higher than ambient
    "v": 2.0,                # Wind speed (m/s)
    "a": 0.05,               # Device width (m)
    "b": 0.0035,             # Unit stage thickness (m)
    "t_ins": 0.01,           # Insulation thickness (m)
    "t_ag": 0.006,           # Aerogel thickness (m)
    "D_a": 3.0e-5,           # Diffusivity of vapor in air (m²/s)
    "h_a": 10.3,             # Convective heat transfer coefficient of air, upward (W/m²K)
    "k": 0.04,               # Thermal conductivity of insulation (W/mK)
    "k_air": 0.026,          # Thermal conductivity of air (W/mK)
    "k_ag": 0.022,           # Thermal conductivity of aerogel (W/mK)
    "Pr": 0.71,              # Prandtl number for air 
    "epsilon": 0.95,         # Emissivity of the solar absorber (CNT: 0.95, SSA: 0.03)
    "h_fg": 2357e3,          # Latent heat of vaporization (J/kg, converted from kJ/kg)
    "M_water": 0.018015,     # Molar mass of water (kg/mol)
    "R_gas": 8.314,          # Universal gas constant (J/(mol*K))
    "sigma": 5.67e-8         # Stefan-Boltzmann constant (W/m²K⁴)
}

In [16]:
# --- Load and process spectral data ---
try:
    data_folder = os.path.join("..", "data")
    file_path = os.path.join(data_folder, "AG_transmittance.csv")
    
    column_names = ['wavelength_m', 'AG2', 'AG4', 'AG6', 'AG8', 'AG12', 'atm']
    AG_data = pd.read_csv(file_path, skiprows=1, names=column_names)

    for col in column_names:
        AG_data[col] = pd.to_numeric(AG_data[col])

    # 3) Calculate the difference between adjacent wavelengths (delta_lambda)
    AG_data['delta_lambda'] = AG_data['wavelength_m'].diff().bfill()

    print("Successfully loaded and processed Aerogel and atmospheric spectral data.")
    
except Exception as e:
    print(f"Error loading or processing AG_transmittance.csv: {e}")
    AG_data = None

Successfully loaded and processed Aerogel and atmospheric spectral data.


In [17]:
# --- Helper Functions ---
def planck_law(wav, T):
    """Calculates spectral radiance using Planck's Law."""
    C1 = 3.7418e-16
    C2 = 1.4388e-2
    return (C1 / (wav**5)) / (np.exp(C2 / (wav * T)) - 1)

def calculate_q_rad(T_f_k, p, spectral_data):
    """
    Calculates q_rad by integrating Planck's law over the spectral data.
    """
    if spectral_data is None: return 0
    T_amb_k = p["T_amb_c"] + 273.15
    
    E_f = planck_law(spectral_data['wavelength_m'], T_f_k)
    E_amb = planck_law(spectral_data['wavelength_m'], T_amb_k)
    
    if p["t_ag"] == 0.002:
        tr = spectral_data['AG2']
    if p["t_ag"] == 0.004:
        tr = spectral_data['AG4']
    if p["t_ag"] == 0.006:
        tr = spectral_data['AG6']
    if p["t_ag"] == 0.008:
        tr = spectral_data['AG8']
    if p["t_ag"] == 0.012:
        tr = spectral_data['AG12']
        
    delta_lambda = spectral_data['delta_lambda']
    epsilon = p["epsilon"]

    if p["condition"] == 'indoor':
        integrand = tr * epsilon * (E_f - E_amb) * delta_lambda
        q_rad = integrand.sum()
    else: # outdoor
        trans_atm = spectral_data['atm']
        integrand = tr * epsilon * (E_f - (1 - trans_atm) * E_amb) * delta_lambda
        q_rad = integrand.sum()       
    return q_rad

def calculate_saturation_concentration(T_k, activity=1.0):
    """
    Calculates water vapor saturation concentration (mol/m³) at a given temperature,
    using August-Roche-Magnus formula, adjusted for the solution's activity.
    """
    T_c = T_k - 273.15
    # P_sat_pure = 610.94 * np.exp((17.625 * T_c) / (T_c + 243.04)) # August-Roche-Magnus formula
    P_sat_pure = 133.32 * 10**8.07131 / 10**(1730.63/(T_c + 233.426)) # Antoine equation
    P_sat_solution = P_sat_pure * activity
    return P_sat_solution / (params["R_gas"] * T_k)

def find_top_surface_temp(T_f_k, p):
    T_amb_k = p["T_amb_c"] + 273.15
    def top_surface_balance(T_top_k):
        q_cond_up = p["k_ag"] * (T_f_k - T_top_k) / p["t_ag"]
        q_conv_out = p["h_a"] * (T_top_k - T_amb_k)
        if p["condition"] == 'indoor':
            q_rad_out = p["epsilon"] * p["sigma"] * (T_top_k**4 - T_amb_k**4)
        if p["condition"] == 'outdoor':
            q_rad_out = (0.0348*(T_top_k-273.15)**2 + 3.75*(T_top_k-273.15) + 113.2) 
        return q_cond_up - (q_conv_out + q_rad_out)
    initial_guess_T_top = (T_f_k + T_amb_k) / 2
    return fsolve(top_surface_balance, initial_guess_T_top)[0]

def solve_stage_backward(q_out, T_b_c, p, is_first_stage=False):
    """
    Solves a single stage backward to find the input heat and front wall temp.
    """
    T_b_k = T_b_c + 273.15

    def find_Tf_from_q_out(T_f_c, q_out_target, T_b_k, p):
        T_f_k = T_f_c + 273.15
        q_cond = p["k_air"] * (T_f_k - T_b_k) / p["b"]
        q_rad_int = 0.95 * p["sigma"]*(T_f_k**4 - T_b_k**4)
        # Use activity=0.98 for evaporating seawater at the front wall
        c_f = calculate_saturation_concentration(T_f_k, activity=0.98)
        # Use activity=1.0 for condensing pure water at the back wall
        c_b = calculate_saturation_concentration(T_b_k, activity=1.0)
        J_evap_molar = p["D_a"] * (c_f - c_b) / p["b"]
        q_evap = (J_evap_molar * p["M_water"]) * p["h_fg"]
        return (q_cond + q_evap + q_rad_int) - q_out_target

    initial_guess_T_f = T_b_c + 5.0
    T_f_c_solved = fsolve(find_Tf_from_q_out, initial_guess_T_f, args=(q_out, T_b_k, p))[0]
    T_f_k_solved = T_f_c_solved + 273.15

    T_top_k_solved = find_top_surface_temp(T_f_k_solved, p)
    T_top_c_solved = T_top_k_solved - 273.15

    T_amb_k = p["T_amb_c"] + 273.15
    R_side = (1 /p["h_a"]) + (p["t_ins"] / p["k"])
    T_avg_side_k = (T_f_k_solved + T_b_k) / 2
    q_side_flux = (T_avg_side_k - T_amb_k) / R_side
    q_side_per_front_area = q_side_flux * (4 * p["b"] / p["a"])

    q_in = q_out + q_side_per_front_area

    if is_first_stage:
        q_rad = calculate_q_rad(T_f_k_solved, p, AG_data)

        R_top = (1 / p["h_a"]) + (p["t_ag"] / p["k_ag"])
        q_cond_up = (T_f_k_solved - T_amb_k) / R_top
        R_top_side = (1 / p["h_a"]) 
        q_cond_side = (((T_f_k_solved + T_top_k_solved) / 2) - T_amb_k) / R_side * (4 * p["t_ag"] / p["a"])

            
        q_in += q_rad + q_cond_up + q_cond_side

    return q_in, T_f_c_solved, T_top_c_solved

In [18]:
# --- Main Multi-Stage Simulation ---
def run_multistage_simulation(n_stages, params):

    q_out_n = params["q_sun"] * params["ref_abs"] * params["Tr"]
    tolerance = 0.1
    max_iterations = 100
    stage_data = []

    print(f"--- Running Multi-Stage Simulation for {n_stages} Stages ---\n")

    for iteration in range(max_iterations):
        current_stage_data = []
        q_out_current = q_out_n
        T_b_current_c = params["T_b_c"]

        for i in range(n_stages, 0, -1):
            is_first = (i == 1)
            q_in_current, T_f_current_c, T_top_current_c = solve_stage_backward(q_out_current, T_b_current_c, params, is_first_stage=is_first)
            current_stage_data.append({"stage": i, "T_f": T_f_current_c, "T_b": T_b_current_c, "T_top": T_top_current_c})
            q_out_current = q_in_current
            T_b_current_c = T_f_current_c

        current_stage_data.reverse()
        q_in_1_calculated = q_out_current # This is now q_in of the first stage
        error = abs(q_in_1_calculated - params["q_sun"] * params["ref_abs"] * params["Tr"])

        print(f"Iteration {iteration + 1}: Calculated q_in,1 = {q_in_1_calculated:.2f} W/m^2, Error = {error:.2f} W/m^2")

        if error < tolerance:
            print("\nConvergence reached!")
            stage_data = current_stage_data
            break

        scaling_factor = params["q_sun"]* params["ref_abs"] * params["Tr"] / q_in_1_calculated
        q_out_n *= scaling_factor
    else:
        print("\nWarning: Maximum iterations reached without convergence.")
        stage_data = current_stage_data

    # --- Final Calculations and Output ---
    print("\n--- Per-Stage Detailed Results ---")
    total_evap_heat = 0
    for data in stage_data:
        T_f_c = data["T_f"]
        T_b_c = data["T_b"]
        T_f_k = T_f_c + 273.15
        T_b_k = T_b_c + 273.15

        # Calculate vapor flux for this stage 
        c_f = calculate_saturation_concentration(T_f_k, activity=0.98)
        c_b = calculate_saturation_concentration(T_b_k, activity=1.0)
        J_evap_molar = params["D_a"] * (c_f - c_b) / params["b"]
        J_evap_mass = J_evap_molar * params["M_water"]
        J_evap_kg_hr = J_evap_mass * 3600
        
        # Calculate heat carried by evaporation
        q_evap = J_evap_mass * params["h_fg"]
        total_evap_heat += q_evap

        if data['stage'] == 1:
            T_top_c = data["T_top"]
            top_temp_str = f"| Top Surface Temp: {T_top_c:.2f}°C "
        else:
            top_temp_str = ""

        print(f"  Stage {data['stage']}: Evap Temp: {data['T_f']:.2f}°C | Condensation Temp: {data['T_b']:.2f}°C | Vapor Flux: {J_evap_kg_hr:.2f} kg/m²/hr")

    print(f"\nTop Surface Temp: { T_top_c:.2f}°C ")
    print("\n--- Overall Performance ---")
    eta_tot = total_evap_heat / params["q_sun"]
    total_vapor_flux_kg_hr = (total_evap_heat / params["h_fg"]) * 3600
    print(f"Total Vapor Mass Flux: {total_vapor_flux_kg_hr:.2f} kg/m²/hr")
    print(f"Total Efficiency (eta_tot): {eta_tot * 100:.2f}%")

In [19]:
# --- Run the Simulation ---
if __name__ == "__main__":
    number_of_stages = 6  
    run_multistage_simulation(number_of_stages, params)

--- Running Multi-Stage Simulation for 6 Stages ---

Iteration 1: Calculated q_in,1 = 1319.44 W/m^2, Error = 406.73 W/m^2
Iteration 2: Calculated q_in,1 = 970.60 W/m^2, Error = 57.89 W/m^2
Iteration 3: Calculated q_in,1 = 922.59 W/m^2, Error = 9.88 W/m^2
Iteration 4: Calculated q_in,1 = 914.44 W/m^2, Error = 1.74 W/m^2
Iteration 5: Calculated q_in,1 = 913.01 W/m^2, Error = 0.31 W/m^2
Iteration 6: Calculated q_in,1 = 912.76 W/m^2, Error = 0.05 W/m^2

Convergence reached!

--- Per-Stage Detailed Results ---
  Stage 1: Evap Temp: 70.72°C | Condensation Temp: 66.12°C | Vapor Flux: 0.94 kg/m²/hr
  Stage 2: Evap Temp: 66.12°C | Condensation Temp: 61.09°C | Vapor Flux: 0.89 kg/m²/hr
  Stage 3: Evap Temp: 61.09°C | Condensation Temp: 55.43°C | Vapor Flux: 0.83 kg/m²/hr
  Stage 4: Evap Temp: 55.43°C | Condensation Temp: 48.86°C | Vapor Flux: 0.78 kg/m²/hr
  Stage 5: Evap Temp: 48.86°C | Condensation Temp: 40.79°C | Vapor Flux: 0.73 kg/m²/hr
  Stage 6: Evap Temp: 40.79°C | Condensation Temp: 30.